In [1]:
import random
from datamatrix import get_init_matrix
from KNN import KNearestNeighbour
import numpy as np

In [2]:
init_matrix = get_init_matrix()
recommender = KNearestNeighbour(k = 50, sim_treshold = 0, measure = "cosine", show = True, mean= True)
data = recommender.init_data(init_matrix)

In [3]:
def evaluation(init_matrix):
    number_of_evaluations = 50
    user_IDs_evaluation = random.sample(range(6040), number_of_evaluations)
    threshold = 3
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    
    for i in range(number_of_evaluations):
        target_user = user_IDs_evaluation[i]
        
        movie_ID_array = np.arange(1, 3953, 1)
        
        target_user_movie_ratings = init_matrix[target_user]
        target_user_rated_movies = np.vstack((movie_ID_array, target_user_movie_ratings))
        
        
        
        target_user_movieID_ratings = []
        for j in range(len(movie_ID_array)):
            if target_user_rated_movies[1][j] != 0:
                target_user_movieID_ratings.append(target_user_rated_movies[0][j])
                target_user_movieID_ratings.append(target_user_rated_movies[1][j])
        
        number_of_evaluated_movies = int(len(target_user_movieID_ratings)*0.2)
        
        target_user_movieID_ratings = np.array(target_user_movieID_ratings)
        target_user_movieID_ratings = target_user_movieID_ratings.reshape(int(len(target_user_movieID_ratings)/2), 2)

        target_user_movieID = target_user_movieID_ratings[:,0]
        target_movies = random.sample(list(target_user_movieID), number_of_evaluated_movies)
        target_movies = np.array(target_movies).astype(int)

        estimated_target_movie_ratings_normalized = recommender.get_ratings(targetid=target_user, targetmovies=target_movies)

        for k in range(len(target_movies)):
            if init_matrix[target_user][target_movies[k]] > threshold and estimated_target_movie_ratings_normalized[k][1] > threshold:
                true_positive += 1
            elif init_matrix[target_user][target_movies[k]] < threshold and estimated_target_movie_ratings_normalized[k][1] > threshold:
                false_positive += 1
                true_negative += 1
            else:
                false_negative += 1

    recall = (true_positive / (true_positive + true_negative))
    precision = (true_positive / (true_positive + false_positive))
    accuracy = ((true_positive + false_negative) / (true_negative + true_positive + false_negative + false_positive))


In [ ]:
evaluation(init_matrix)

Shape of targetid_movies (30,) 

Shape of targetid_ratings (30,) 

Shape of only_targets (6040, 30) 

Shape of no_empty_ids (5003, 30) 

Shape of not_user (5002, 30) 

Shape of final_matrix is: (5002, 30) 

sim_and_id 5002 (5002, 2)
Shape of sim_and_id: (5002, 2) 

Example of sim_and_id is: [[ 1.          0.06256449]
 [ 2.          0.04436492]
 [ 3.          0.70817516]
 [ 4.          1.        ]
 [ 5.         -0.39578439]] 

neighbour [[4.567e+03 1.000e+00]
 [3.931e+03 1.000e+00]
 [2.100e+01 1.000e+00]
 [3.867e+03 1.000e+00]
 [3.500e+03 1.000e+00]
 [1.825e+03 1.000e+00]
 [4.176e+03 1.000e+00]
 [4.733e+03 1.000e+00]
 [2.307e+03 1.000e+00]
 [4.197e+03 1.000e+00]
 [3.177e+03 1.000e+00]
 [5.041e+03 1.000e+00]
 [1.270e+02 1.000e+00]
 [8.680e+02 1.000e+00]
 [2.974e+03 1.000e+00]
 [3.123e+03 1.000e+00]
 [2.989e+03 1.000e+00]
 [1.041e+03 1.000e+00]
 [4.297e+03 1.000e+00]
 [3.651e+03 1.000e+00]
 [4.939e+03 1.000e+00]
 [3.052e+03 1.000e+00]
 [3.055e+03 1.000e+00]
 [4.496e+03 1.000e+00]
 [7.780e

sim_and_id 5763 (5763, 2)
Shape of sim_and_id: (5763, 2) 

Example of sim_and_id is: [[ 1.         -0.21334449]
 [ 2.         -0.32061648]
 [ 3.         -0.32831605]
 [ 4.          0.87815294]
 [ 5.         -0.10972058]] 

neighbour [[6.940e+02 1.000e+00]
 [4.753e+03 1.000e+00]
 [4.778e+03 1.000e+00]
 [1.247e+03 1.000e+00]
 [5.048e+03 1.000e+00]
 [1.610e+03 1.000e+00]
 [4.247e+03 1.000e+00]
 [3.377e+03 1.000e+00]
 [3.613e+03 1.000e+00]
 [3.797e+03 1.000e+00]
 [4.936e+03 1.000e+00]
 [1.691e+03 1.000e+00]
 [4.495e+03 1.000e+00]
 [4.401e+03 1.000e+00]
 [3.084e+03 1.000e+00]
 [2.210e+02 1.000e+00]
 [4.070e+02 1.000e+00]
 [4.176e+03 1.000e+00]
 [1.913e+03 1.000e+00]
 [2.306e+03 1.000e+00]
 [5.920e+02 1.000e+00]
 [5.210e+02 1.000e+00]
 [3.421e+03 1.000e+00]
 [5.735e+03 1.000e+00]
 [9.470e+02 1.000e+00]
 [5.098e+03 1.000e+00]
 [6.700e+02 1.000e+00]
 [3.124e+03 1.000e+00]
 [3.427e+03 1.000e+00]
 [3.429e+03 1.000e+00]
 [1.713e+03 1.000e+00]
 [4.503e+03 1.000e+00]
 [1.554e+03 1.000e+00]
 [2.910e

sim_and_id 6005 (6005, 2)
Shape of sim_and_id: (6005, 2) 

Example of sim_and_id is: [[ 1.          0.25748291]
 [ 2.          0.52234555]
 [ 3.         -0.21382841]
 [ 4.         -0.22781233]
 [ 5.         -0.01305227]] 

neighbour [[4.811e+03 1.000e+00]
 [5.291e+03 1.000e+00]
 [5.028e+03 1.000e+00]
 [4.119e+03 1.000e+00]
 [4.170e+02 1.000e+00]
 [5.051e+03 1.000e+00]
 [4.129e+03 1.000e+00]
 [4.200e+02 1.000e+00]
 [2.442e+03 1.000e+00]
 [2.417e+03 1.000e+00]
 [1.337e+03 1.000e+00]
 [4.470e+02 1.000e+00]
 [4.191e+03 1.000e+00]
 [4.192e+03 1.000e+00]
 [2.361e+03 1.000e+00]
 [8.740e+02 1.000e+00]
 [4.102e+03 1.000e+00]
 [5.965e+03 1.000e+00]
 [5.238e+03 1.000e+00]
 [4.255e+03 1.000e+00]
 [4.262e+03 1.000e+00]
 [2.150e+02 1.000e+00]
 [4.991e+03 1.000e+00]
 [3.486e+03 1.000e+00]
 [5.417e+03 1.000e+00]
 [4.282e+03 1.000e+00]
 [4.283e+03 1.000e+00]
 [1.042e+03 1.000e+00]
 [4.106e+03 1.000e+00]
 [3.840e+02 1.000e+00]
 [4.348e+03 1.000e+00]
 [5.066e+03 1.000e+00]
 [3.662e+03 1.000e+00]
 [2.960e

Shape of final_matrix is: (6026, 280) 

sim_and_id 6026 (6026, 2)
Shape of sim_and_id: (6026, 2) 

Example of sim_and_id is: [[ 1.          0.11210829]
 [ 2.         -0.04824415]
 [ 3.          0.20670688]
 [ 4.          0.80950708]
 [ 5.          0.45690285]] 

neighbour [[5.33400000e+03 1.00000000e+00]
 [1.27000000e+03 1.00000000e+00]
 [4.59700000e+03 1.00000000e+00]
 [3.24500000e+03 1.00000000e+00]
 [4.41300000e+03 1.00000000e+00]
 [1.89300000e+03 1.00000000e+00]
 [4.05600000e+03 1.00000000e+00]
 [1.88800000e+03 1.00000000e+00]
 [3.04300000e+03 1.00000000e+00]
 [4.52700000e+03 1.00000000e+00]
 [4.23000000e+03 1.00000000e+00]
 [4.42100000e+03 1.00000000e+00]
 [4.99100000e+03 1.00000000e+00]
 [3.95900000e+03 1.00000000e+00]
 [4.00800000e+03 1.00000000e+00]
 [1.71000000e+02 1.00000000e+00]
 [2.99200000e+03 1.00000000e+00]
 [7.82000000e+02 1.00000000e+00]
 [3.54200000e+03 1.00000000e+00]
 [1.22600000e+03 1.00000000e+00]
 [2.33600000e+03 1.00000000e+00]
 [1.24000000e+03 1.00000000e+00]
 

sim_and_id 5949 (5949, 2)
Shape of sim_and_id: (5949, 2) 

Example of sim_and_id is: [[ 1.         -0.09700977]
 [ 2.          0.27642272]
 [ 3.         -0.26000064]
 [ 4.         -1.        ]
 [ 5.          0.27028074]] 

neighbour [[2.284e+03 1.000e+00]
 [1.443e+03 1.000e+00]
 [1.708e+03 1.000e+00]
 [5.822e+03 1.000e+00]
 [3.877e+03 1.000e+00]
 [3.803e+03 1.000e+00]
 [3.376e+03 1.000e+00]
 [1.600e+02 1.000e+00]
 [1.580e+02 1.000e+00]
 [5.695e+03 1.000e+00]
 [4.129e+03 1.000e+00]
 [5.696e+03 1.000e+00]
 [3.802e+03 1.000e+00]
 [4.110e+03 1.000e+00]
 [1.417e+03 1.000e+00]
 [5.909e+03 1.000e+00]
 [5.000e+01 1.000e+00]
 [5.391e+03 1.000e+00]
 [1.970e+03 1.000e+00]
 [4.100e+03 1.000e+00]
 [5.055e+03 1.000e+00]
 [1.424e+03 1.000e+00]
 [2.710e+03 1.000e+00]
 [5.036e+03 1.000e+00]
 [3.867e+03 1.000e+00]
 [4.157e+03 1.000e+00]
 [8.710e+02 1.000e+00]
 [3.734e+03 1.000e+00]
 [2.875e+03 1.000e+00]
 [2.867e+03 1.000e+00]
 [1.351e+03 1.000e+00]
 [2.827e+03 1.000e+00]
 [2.816e+03 1.000e+00]
 [4.623e

...

Shape of final_matrix is: (5757, 196) 

sim_and_id 5757 (5757, 2)
Shape of sim_and_id: (5757, 2) 

Example of sim_and_id is: [[ 1.         -0.30996893]
 [ 2.          0.25498992]
 [ 3.         -0.08549865]
 [ 4.          0.19028351]
 [ 5.          0.28068879]] 

neighbour [[4.299e+03 1.000e+00]
 [1.462e+03 1.000e+00]
 [2.970e+03 1.000e+00]
 [1.412e+03 1.000e+00]
 [2.317e+03 1.000e+00]
 [6.530e+02 1.000e+00]
 [1.432e+03 1.000e+00]
 [3.593e+03 1.000e+00]
 [4.860e+02 1.000e+00]
 [2.130e+03 1.000e+00]
 [7.000e+00 1.000e+00]
 [2.548e+03 1.000e+00]
 [4.463e+03 1.000e+00]
 [1.602e+03 1.000e+00]
 [2.953e+03 1.000e+00]
 [1.347e+03 1.000e+00]
 [4.321e+03 1.000e+00]
 [1.158e+03 1.000e+00]
 [1.157e+03 1.000e+00]
 [3.630e+02 1.000e+00]
 [1.153e+03 1.000e+00]
 [1.715e+03 1.000e+00]
 [3.076e+03 1.000e+00]
 [4.840e+02 1.000e+00]
 [9.980e+02 1.000e+00]
 [3.610e+02 1.000e+00]
 [1.160e+03 1.000e+00]
 [9.970e+02 1.000e+00]
 [3.968e+03 1.000e+00]
 [3.969e+03 1.000e+00]
 [3.334e+03 1.000e+00]
 [1.529e+03 1.

Shape of final_matrix is: (6005, 153) 

sim_and_id 6005 (6005, 2)
Shape of sim_and_id: (6005, 2) 

Example of sim_and_id is: [[ 1.          0.33491317]
 [ 2.          0.19406989]
 [ 3.         -0.29836395]
 [ 4.          0.90632848]
 [ 5.          0.01144141]] 

neighbour [[4.912e+03 1.000e+00]
 [3.803e+03 1.000e+00]
 [5.069e+03 1.000e+00]
 [4.324e+03 1.000e+00]
 [2.582e+03 1.000e+00]
 [1.777e+03 1.000e+00]
 [3.788e+03 1.000e+00]
 [4.464e+03 1.000e+00]
 [3.251e+03 1.000e+00]
 [1.346e+03 1.000e+00]
 [2.289e+03 1.000e+00]
 [2.291e+03 1.000e+00]
 [3.765e+03 1.000e+00]
 [4.442e+03 1.000e+00]
 [3.040e+02 1.000e+00]
 [3.170e+02 1.000e+00]
 [5.207e+03 1.000e+00]
 [5.125e+03 1.000e+00]
 [5.080e+03 1.000e+00]
 [3.282e+03 1.000e+00]
 [5.221e+03 1.000e+00]
 [5.067e+03 1.000e+00]
 [2.159e+03 1.000e+00]
 [4.544e+03 1.000e+00]
 [4.548e+03 1.000e+00]
 [3.321e+03 1.000e+00]
 [2.086e+03 1.000e+00]
 [4.991e+03 1.000e+00]
 [1.573e+03 1.000e+00]
 [4.955e+03 1.000e+00]
 [1.640e+02 1.000e+00]
 [4.632e+03 1.

sim_and_id 4559 (4559, 2)
Shape of sim_and_id: (4559, 2) 

Example of sim_and_id is: [[ 1.         -0.08380143]
 [ 2.         -0.09756267]
 [ 3.         -0.62867284]
 [ 5.          0.75425141]
 [ 6.          0.62603352]] 

neighbour [[2.306e+03 1.000e+00]
 [5.365e+03 1.000e+00]
 [1.971e+03 1.000e+00]
 [2.747e+03 1.000e+00]
 [4.857e+03 1.000e+00]
 [5.559e+03 1.000e+00]
 [3.174e+03 1.000e+00]
 [2.904e+03 1.000e+00]
 [9.460e+02 1.000e+00]
 [4.489e+03 1.000e+00]
 [6.070e+02 1.000e+00]
 [3.127e+03 1.000e+00]
 [5.869e+03 1.000e+00]
 [5.875e+03 1.000e+00]
 [8.980e+02 1.000e+00]
 [4.209e+03 1.000e+00]
 [3.917e+03 1.000e+00]
 [2.692e+03 1.000e+00]
 [4.890e+02 1.000e+00]
 [5.913e+03 1.000e+00]
 [4.194e+03 1.000e+00]
 [5.257e+03 1.000e+00]
 [4.552e+03 1.000e+00]
 [4.720e+02 1.000e+00]
 [4.753e+03 1.000e+00]
 [6.400e+02 1.000e+00]
 [9.970e+02 1.000e+00]
 [3.392e+03 1.000e+00]
 [7.840e+02 1.000e+00]
 [3.306e+03 1.000e+00]
 [2.970e+03 1.000e+00]
 [2.892e+03 1.000e+00]
 [4.990e+03 1.000e+00]
 [1.817e

sim_and_id 5997 (5997, 2)
Shape of sim_and_id: (5997, 2) 

Example of sim_and_id is: [[ 1.         -0.60573242]
 [ 2.          0.39794747]
 [ 3.         -0.22363302]
 [ 4.         -0.36699885]
 [ 5.          0.37413947]] 

neighbour [[8.330e+02 1.000e+00]
 [4.273e+03 1.000e+00]
 [4.175e+03 1.000e+00]
 [2.582e+03 1.000e+00]
 [3.595e+03 1.000e+00]
 [9.420e+02 1.000e+00]
 [1.927e+03 1.000e+00]
 [3.623e+03 1.000e+00]
 [2.364e+03 1.000e+00]
 [5.533e+03 1.000e+00]
 [3.633e+03 1.000e+00]
 [2.155e+03 1.000e+00]
 [5.919e+03 1.000e+00]
 [5.979e+03 1.000e+00]
 [4.804e+03 1.000e+00]
 [1.893e+03 1.000e+00]
 [1.568e+03 1.000e+00]
 [4.470e+02 1.000e+00]
 [4.061e+03 1.000e+00]
 [4.464e+03 1.000e+00]
 [4.031e+03 1.000e+00]
 [2.422e+03 1.000e+00]
 [2.150e+03 1.000e+00]
 [4.178e+03 1.000e+00]
 [5.391e+03 1.000e+00]
 [2.381e+03 1.000e+00]
 [1.213e+03 1.000e+00]
 [2.111e+03 1.000e+00]
 [4.549e+03 1.000e+00]
 [4.340e+03 1.000e+00]
 [1.580e+02 1.000e+00]
 [1.710e+02 1.000e+00]
 [4.282e+03 1.000e+00]
 [5.772e

sim_and_id 5934 (5934, 2)
Shape of sim_and_id: (5934, 2) 

Example of sim_and_id is: [[ 1.         -0.21004847]
 [ 2.         -0.32518237]
 [ 3.          0.07827581]
 [ 4.         -0.057456  ]
 [ 5.          0.89826027]] 

neighbour [[4.649e+03 1.000e+00]
 [4.651e+03 1.000e+00]
 [7.840e+02 1.000e+00]
 [2.759e+03 1.000e+00]
 [5.176e+03 1.000e+00]
 [4.434e+03 1.000e+00]
 [4.648e+03 1.000e+00]
 [5.203e+03 1.000e+00]
 [5.354e+03 1.000e+00]
 [3.802e+03 1.000e+00]
 [3.897e+03 1.000e+00]
 [5.789e+03 1.000e+00]
 [3.366e+03 1.000e+00]
 [3.893e+03 1.000e+00]
 [1.785e+03 1.000e+00]
 [1.070e+03 1.000e+00]
 [5.720e+02 1.000e+00]
 [4.025e+03 1.000e+00]
 [2.381e+03 1.000e+00]
 [9.800e+01 1.000e+00]
 [1.808e+03 1.000e+00]
 [5.529e+03 1.000e+00]
 [1.708e+03 1.000e+00]
 [4.664e+03 1.000e+00]
 [5.467e+03 1.000e+00]
 [4.660e+03 1.000e+00]
 [9.060e+02 1.000e+00]
 [4.413e+03 1.000e+00]
 [4.074e+03 1.000e+00]
 [5.550e+02 1.000e+00]
 [7.700e+01 1.000e+00]
 [3.780e+02 1.000e+00]
 [4.419e+03 1.000e+00]
 [4.421e